In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib.patches import Circle, Wedge
from matplotlib.colors import hsv_to_rgb
from matplotlib.lines import Line2D
from shapely.geometry import Point
from shapely.ops import unary_union
from matplotlib.animation import FuncAnimation
from ipywidgets import interact_manual, IntSlider, FloatSlider

In [ ]:
ALG_PARAMS = {
    "n_points": 50,
    "canvas_width": 150,
    "canvas_height": 100,
    "scan_radius": 10,
    "min_points": 3,
}

In [ ]:
POINT_STYLE = {
    "size": 30,
    "core_color": "darkred",
    "reachable_color": "lightcoral",
    "unclassified_color": "gray",
    "noise_color": "black",
}

CIRCLE_STYLE = {"border_color": "black", "border_width": 1.0}
LINK_STYLE = {"color": "green", "width": 1.0, "linestyle": "--"}
REGION_STYLE = {"color": "black", "width": 2.0, "label_fontsize": 20}

In [ ]:
def find_core_points(points, scan_radius, min_points):
    core_indices = []
    points_in_circle = []
    for i, p in enumerate(points):
        neighbors = np.where(np.linalg.norm(points - p, axis=1) <= scan_radius)[0]
        if len(neighbors) >= min_points:
            core_indices.append(i)
            points_in_circle.append(set(neighbors))
        else:
            points_in_circle.append(set([i]))
    return points[core_indices], core_indices, points_in_circle


def build_core_graph(core_indices, points_in_circle):
    G = nx.Graph()
    for idx in range(len(core_indices)):
        G.add_node(idx)
    for i, ci in enumerate(core_indices):
        for j, cj in enumerate(core_indices):
            if i >= j:
                continue
            if len(points_in_circle[ci] & points_in_circle[cj]) > 0:
                G.add_edge(i, j)
    return G


def find_reachable_and_noise(points, core_indices, G, core_points, scan_radius):
    reachable = set()
    for component in nx.connected_components(G):
        circles = [
            Point(core_points[i]).buffer(scan_radius, resolution=64) for i in component
        ]
        union_shape = unary_union(circles)
        for i, p in enumerate(points):
            if i not in core_indices and union_shape.contains(Point(p)):
                reachable.add(i)
    noise = set(range(len(points))) - set(core_indices) - reachable
    return reachable, noise

In [ ]:
def base_axis(ax, canvas_width, canvas_height):
    ax.set_xlim(0, canvas_width)
    ax.set_ylim(0, canvas_height)
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])


def plot_all_points(ax, points, core_indices=None, reachable=None, noise=None):
    ax.scatter(
        points[:, 0],
        points[:, 1],
        s=POINT_STYLE["size"],
        color=POINT_STYLE["unclassified_color"],
        zorder=1,
    )
    if core_indices:
        ax.scatter(
            points[core_indices, 0],
            points[core_indices, 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["core_color"],
            zorder=4,
        )
    if reachable:
        ax.scatter(
            points[list(reachable), 0],
            points[list(reachable), 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["reachable_color"],
            zorder=4,
        )
    if noise:
        ax.scatter(
            points[list(noise), 0],
            points[list(noise), 1],
            s=POINT_STYLE["size"],
            color=POINT_STYLE["noise_color"],
            marker="x",
            zorder=5,
        )


def plot_core_circles(ax, core_points, scan_radius):
    for p in core_points:
        ax.add_patch(
            Circle(
                p,
                scan_radius,
                fill=False,
                edgecolor=CIRCLE_STYLE["border_color"],
                linewidth=CIRCLE_STYLE["border_width"],
                zorder=3,
            )
        )


def plot_core_links(ax, G, core_points):
    for i, j in G.edges():
        p1, p2 = core_points[i], core_points[j]
        ax.plot(
            [p1[0], p2[0]],
            [p1[1], p2[1]],
            color=LINK_STYLE["color"],
            linewidth=LINK_STYLE["width"],
            linestyle=LINK_STYLE["linestyle"],
            zorder=2,
        )


def plot_connected_regions(ax, G, core_points, scan_radius, fill=False, alpha=0.15):
    """
    Plot connected regions of core points.

    Parameters:
    - ax: matplotlib Axes object
    - G: networkx Graph of core connections
    - core_points: array of core point coordinates
    - scan_radius: radius of core circles
    - fill: if True, fill the region with semi-transparent color
    - alpha: transparency level for filled regions (0-1)
    """
    components = list(nx.connected_components(G))
    for idx,component in enumerate(components):
        circles = [
            Point(core_points[i]).buffer(scan_radius, resolution=64) for i in component
        ]
        union_shape = unary_union(circles)
        polygons = (
            [union_shape]
            if union_shape.geom_type == "Polygon"
            else list(union_shape.geoms)
        )
        color = tuple(hsv_to_rgb([idx / len(components), 0.8, 0.9]))
        for poly in polygons:
            x, y = poly.exterior.xy
            if fill:
                # Fill the region with semi-transparent color
                ax.fill(
                    x,
                    y,
                    color=color,
                    alpha=alpha,
                    zorder=1,
                )
            # Place the index label at the representative point
            label_pt = poly.representative_point()
            rgb = np.array(color[:3])
            luminance = 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]
            text_color = "white" if luminance < 0.5 else "black"
            ax.text(
                label_pt.x,
                label_pt.y,
                str(idx + 1),
                fontsize=REGION_STYLE["label_fontsize"],
                fontweight="bold",
                ha="center",
                va="center",
                color=text_color,
                zorder=7,
            )
            # Draw the border
            ax.plot(
                x,
                y,
                color=REGION_STYLE["color"],
                linewidth=REGION_STYLE["width"],
                zorder=6,
            )


def add_dynamic_legend(
    ax,
    show_core_indices: bool = False,
    show_reachable: bool = False,
    show_noise: bool = False,
    show_circles: bool = False,
    show_links: bool = False,
    show_regions: bool = False,
):
    handles = []

    handles.append(
        Line2D(
            [0],
            [0],
            marker="o",
            color="w",
            label="Unclassified",
            markerfacecolor=POINT_STYLE["unclassified_color"],
            markersize=8,
        )
    )
    if show_core_indices:
        handles.append(
            Line2D(
                [0],
                [0],
                marker="o",
                color="w",
                label="Core",
                markerfacecolor=POINT_STYLE["core_color"],
                markersize=8,
            )
        )
    if show_reachable:
        handles.append(
            Line2D(
                [0],
                [0],
                marker="o",
                color="w",
                label="Reachable",
                markerfacecolor=POINT_STYLE["reachable_color"],
                markersize=8,
            )
        )
    if show_noise:
        handles.append(
            Line2D(
                [0], [0], marker="x", color=POINT_STYLE["noise_color"], label="Noise"
            )
        )
    if show_circles:
        handles.append(
            Line2D(
                [0],
                [0],
                color=CIRCLE_STYLE["border_color"],
                lw=CIRCLE_STYLE["border_width"],
                label="Core Circle",
            )
        )
    if show_links:
        handles.append(
            Line2D(
                [0],
                [0],
                color=LINK_STYLE["color"],
                lw=LINK_STYLE["width"],
                linestyle=LINK_STYLE["linestyle"],
                label="Core Link",
            )
        )
    if show_regions:
        handles.append(
            Line2D(
                [0],
                [0],
                color=REGION_STYLE["color"],
                lw=REGION_STYLE["width"],
                label="Connected Region",
            )
        )

    if len(handles) > 0:
        ax.legend(handles=handles, loc="upper right")

In [ ]:
animations = []  # global list to keep animation objects alive

In [ ]:
# Radar animation configuration
RADAR_STYLE = {
    "circle_fill": "lightgreen",
    "circle_color": "lightgreen",
    "circle_linewidth": 2,
    "wedge_color": "green",
    "wedge_base_alpha": 0.4,
    "animation_interval": 5,  # milliseconds between frames
    "radar_alpha_base": 0.6,  # base alpha for radar circles (used for scanning and fading)
    "radar_alpha_step": 0.1,  # alpha reduction per concurrent scanner
}

# Radar animation parameters
RADAR_PARAMS = {
    "sweep_angle": 60,  # angular width of wedge (degrees)
    "sweep_speed": 5,  # rotation speed per frame (degrees)
    "n_fade": 10,  # number of gradient wedges for fading effect
    "concurrent_scans": 3,  # number of points to scan simultaneously
    "random_start_angle": True,  # whether to start each scan at a random angle
    "fade_frames": 15,  # number of frames for radar circle to fade out after scan
}

In [ ]:
class RadarScanner:
    """Encapsulates radar scanning state and logic."""

    def __init__(self, points, min_points, scan_radius):
        self.points = points
        self.min_points = min_points
        self.scan_radius = scan_radius
        self.n_points = len(points)

        # Tracking confirmed points
        self.confirmed_core_indices = set()
        self.confirmed_reachable_indices = set()
        self.core_circles = {}

        # Tracking scanned points to avoid re-scanning
        self.scanned_points = set()  # Set of point indices that have been scanned
        self.scanning_points = (
            {}
        )  # {scanner_idx: point_idx} - currently scanning points
        self.fading_points = (
            {}
        )  # {scanner_idx: {'point_idx': int, 'is_core': bool}} - points in fade stage

    def get_point_colors(self):
        """Generate color list for all points based on current classification."""
        return [
            (
                POINT_STYLE["core_color"]
                if idx in self.confirmed_core_indices
                else (
                    POINT_STYLE["reachable_color"]
                    if idx in self.confirmed_reachable_indices
                    else POINT_STYLE["unclassified_color"]
                )
            )
            for idx in range(self.n_points)
        ]

    def find_neighbors(self, center_idx):
        """Find all neighbors within scan radius of given point."""
        center = self.points[center_idx]
        distances = np.linalg.norm(self.points - center, axis=1)
        return np.where(distances <= self.scan_radius)[0]

    def classify_point_as_core(self, center_idx, neighbors):
        """
        Classify a point as core and update reachable points.
        Thread-safe for concurrent scanning: ensures no conflicts when multiple
        scanners complete simultaneously.
        """
        # First, mark as core point
        self.confirmed_core_indices.add(center_idx)

        # Remove this point from reachable if it was there
        self.confirmed_reachable_indices.discard(center_idx)

        # Update reachable points (exclude ALL confirmed core points)
        # This prevents race conditions where a neighbor might have just become core
        new_reachable = set(neighbors) - self.confirmed_core_indices

        # Also remove any neighbors that are already core from reachable set
        # This handles the case where neighbors became core in parallel
        self.confirmed_reachable_indices -= self.confirmed_core_indices

        # Now safely add new reachable points
        self.confirmed_reachable_indices.update(new_reachable)

    def process_scan_completion(self, center_idx, ax):
        """Process logic when a full 360° scan is completed for a point."""
        neighbors = self.find_neighbors(center_idx)

        is_core = len(neighbors) >= self.min_points

        if is_core:
            self.classify_point_as_core(center_idx, neighbors)

            # Create and store core circle visualization
            center = self.points[center_idx]
            core_circle = Circle(
                center,
                self.scan_radius,
                fill=False,
                edgecolor=CIRCLE_STYLE["border_color"],
                linewidth=CIRCLE_STYLE["border_width"],
                zorder=3,
            )
            ax.add_patch(core_circle)
            self.core_circles[center_idx] = core_circle

        return is_core

In [ ]:
def create_radar_wedges(scan_radius, sweep_angle, n_fade):
    """Create gradient wedges for radar sweep effect."""
    wedges = []
    for i in range(n_fade):
        wedge_angle = sweep_angle * (i + 1) / n_fade
        wedge_alpha = (i + 1) / n_fade * RADAR_STYLE["wedge_base_alpha"]
        wedge = Wedge(
            (0, 0),
            scan_radius,
            0,
            wedge_angle,
            facecolor=RADAR_STYLE["wedge_color"],
            alpha=wedge_alpha,
        )
        wedges.append(wedge)
    return wedges


def update_wedge_positions(wedges, center, theta_start, sweep_angle, n_fade):
    """Update positions and angles of all radar wedges."""
    for i, wedge in enumerate(wedges):
        wedge.center = center
        wedge.set_theta1(theta_start)
        wedge_angle = sweep_angle * (i + 1) / n_fade
        wedge.set_theta2(theta_start + wedge_angle)

In [ ]:
def radar_animation(
    ax,
    points,
    min_points,
    scan_radius,
    sweep_angle=None,
    sweep_speed=None,
    n_fade=None,
    concurrent_scans=None,
    random_start_angle=None,
    fade_frames=None,
):
    """
    Animate radar scan for points, progressively identifying core points.
    Supports concurrent scanning of multiple points simultaneously.

    Parameters:
    - ax: matplotlib Axes object to draw on
    - points: numpy array (N, 2), coordinates of all points
    - min_points: minimum neighbors to qualify as core point
    - scan_radius: scanning radius
    - sweep_angle: angular width of wedge (degrees), defaults to RADAR_PARAMS["sweep_angle"]
    - sweep_speed: rotation speed per frame (degrees), defaults to RADAR_PARAMS["sweep_speed"]
    - n_fade: number of gradient wedges for fading effect, defaults to RADAR_PARAMS["n_fade"]
    - concurrent_scans: number of points to scan simultaneously, defaults to RADAR_PARAMS["concurrent_scans"]
    - random_start_angle: whether to randomize start angle, defaults to RADAR_PARAMS["random_start_angle"]
    - fade_frames: frames for radar circle fade out, defaults to RADAR_PARAMS["fade_frames"]

    Returns:
    - ani: FuncAnimation object
    """
    # Use configuration defaults if not specified
    sweep_angle = (
        sweep_angle if sweep_angle is not None else RADAR_PARAMS["sweep_angle"]
    )
    sweep_speed = (
        sweep_speed if sweep_speed is not None else RADAR_PARAMS["sweep_speed"]
    )
    n_fade = n_fade if n_fade is not None else RADAR_PARAMS["n_fade"]
    concurrent_scans = (
        concurrent_scans
        if concurrent_scans is not None
        else RADAR_PARAMS["concurrent_scans"]
    )
    random_start_angle = (
        random_start_angle
        if random_start_angle is not None
        else RADAR_PARAMS["random_start_angle"]
    )
    fade_frames = (
        fade_frames if fade_frames is not None else RADAR_PARAMS["fade_frames"]
    )

    fig = ax.figure

    # Initialize scanner state
    scanner = RadarScanner(points, min_points, scan_radius)

    # Generate random start angles for each point if enabled
    start_angles = {}
    if random_start_angle:
        for i in range(len(points)):
            start_angles[i] = np.random.uniform(0, 360)
    else:
        for i in range(len(points)):
            start_angles[i] = 0

    # Initialize scanner state
    scanner = RadarScanner(points, min_points, scan_radius)

    # Initialize point visualization
    initial_colors = scanner.get_point_colors()
    scatter_plot = ax.scatter(
        points[:, 0],
        points[:, 1],
        s=POINT_STYLE["size"],
        color=initial_colors,
        zorder=4,
    )

    # Create radar elements for each concurrent scanner
    radar_elements = []
    for i in range(concurrent_scans):
        # Create radar circle with slight color variation
        base_alpha = RADAR_STYLE["radar_alpha_base"] - (
            i * RADAR_STYLE["radar_alpha_step"]
        )

        radar_circle = Circle(
            (0, 0),
            scan_radius,
            fill=True,
            facecolor=RADAR_STYLE["circle_fill"],
            edgecolor=RADAR_STYLE["circle_color"],
            lw=RADAR_STYLE["circle_linewidth"],
            alpha=base_alpha,
            zorder=2,
        )
        ax.add_patch(radar_circle)

        # Create wedges for this scanner
        wedges = create_radar_wedges(scan_radius, sweep_angle, n_fade)
        for wedge in wedges:
            # Vary wedge opacity for each concurrent scanner
            wedge.set_alpha(wedge.get_alpha() * (1 - i * 0.15))
            ax.add_patch(wedge)

        radar_elements.append(
            {
                "circle": radar_circle,
                "wedges": wedges,
                "base_alpha": base_alpha,  # Store base alpha for this scanner
            }
        )

    # Animation parameters
    frames_per_point = int(360 / sweep_speed)
    # Add extra frames for fade effect
    frames_per_point_total = frames_per_point + fade_frames
    # Calculate total frames for concurrent scanning
    total_scan_batches = int(np.ceil(scanner.n_points / concurrent_scans))
    total_frames = total_scan_batches * frames_per_point_total

    def update_frame(frame):
        """Update function called for each animation frame."""
        batch_idx = frame // frames_per_point_total
        frame_in_cycle = frame % frames_per_point_total
        frame_in_scan = min(frame_in_cycle, frames_per_point - 1)

        is_scanning = frame_in_cycle < frames_per_point
        is_scan_complete = frame_in_cycle == frames_per_point - 1
        is_fading = frame_in_cycle >= frames_per_point
        fade_progress = (
            (frame_in_cycle - frames_per_point) / fade_frames if is_fading else 0
        )

        # Update each concurrent scanner
        active_artists = [scatter_plot]

        for scanner_idx in range(concurrent_scans):
            # Calculate which point this scanner is processing
            point_idx = batch_idx * concurrent_scans + scanner_idx

            # Skip if we've run out of points
            if point_idx >= scanner.n_points:
                # Hide this scanner's elements
                radar_elements[scanner_idx]["circle"].set_visible(False)
                for wedge in radar_elements[scanner_idx]["wedges"]:
                    wedge.set_visible(False)
                continue

            # Check if this point has already been scanned - SKIP if so
            if point_idx in scanner.scanned_points:
                # Hide this scanner's elements
                radar_elements[scanner_idx]["circle"].set_visible(False)
                for wedge in radar_elements[scanner_idx]["wedges"]:
                    wedge.set_visible(False)
                continue

            # Get the start angle for this point
            point_start_angle = start_angles[point_idx]
            theta_start = point_start_angle + frame_in_scan * sweep_speed

            # Update radar position
            current_center = points[point_idx]
            radar_elements[scanner_idx]["circle"].center = current_center

            if is_scanning:
                # Normal scanning mode
                # Mark this point as currently being scanned
                scanner.scanning_points[scanner_idx] = point_idx

                radar_elements[scanner_idx]["circle"].set_visible(True)
                for wedge in radar_elements[scanner_idx]["wedges"]:
                    wedge.set_visible(True)

                # Update wedge positions
                update_wedge_positions(
                    radar_elements[scanner_idx]["wedges"],
                    current_center,
                    theta_start,
                    sweep_angle,
                    n_fade,
                )

                # Use default color during scan
                radar_elements[scanner_idx]["circle"].set_facecolor(
                    RADAR_STYLE["circle_fill"]
                )
                radar_elements[scanner_idx]["circle"].set_edgecolor(
                    RADAR_STYLE["circle_color"]
                )
                radar_elements[scanner_idx]["circle"].set_alpha(
                    radar_elements[scanner_idx]["base_alpha"]
                )

            elif is_fading:
                # Fading mode after scan complete
                # Hide wedges during fade
                for wedge in radar_elements[scanner_idx]["wedges"]:
                    wedge.set_visible(False)

                # Initialize fading state if not already done
                if scanner_idx not in scanner.fading_points:
                    is_core = point_idx in scanner.confirmed_core_indices
                    scanner.fading_points[scanner_idx] = {
                        "point_idx": point_idx,
                        "is_core": is_core,
                    }

                is_core = point_idx in scanner.confirmed_core_indices
                base_alpha = radar_elements[scanner_idx]["base_alpha"]

                if is_core:
                    # Core point: turn red and fade out
                    radar_elements[scanner_idx]["circle"].set_facecolor("red")
                    radar_elements[scanner_idx]["circle"].set_edgecolor("darkred")
                    radar_elements[scanner_idx]["circle"].set_alpha(
                        (1 - fade_progress) * base_alpha
                    )
                    radar_elements[scanner_idx]["circle"].set_visible(True)
                else:
                    # Non-core point: keep green color and fade out quickly
                    radar_elements[scanner_idx]["circle"].set_facecolor(
                        RADAR_STYLE["circle_fill"]
                    )
                    radar_elements[scanner_idx]["circle"].set_edgecolor(
                        RADAR_STYLE["circle_color"]
                    )
                    radar_elements[scanner_idx]["circle"].set_alpha(
                        (1 - fade_progress) * base_alpha
                    )
                    radar_elements[scanner_idx]["circle"].set_visible(
                        fade_progress < 0.8
                    )

                # Mark as completely scanned when fade is complete
                if fade_progress >= 1.0:
                    scanner.scanned_points.add(point_idx)
                    if scanner_idx in scanner.scanning_points:
                        del scanner.scanning_points[scanner_idx]
                    if scanner_idx in scanner.fading_points:
                        del scanner.fading_points[scanner_idx]

            # Check if scan is complete (happens once at end of 360° scan)
            if is_scan_complete:
                is_core = scanner.process_scan_completion(point_idx, ax)

            # Add to active artists
            if radar_elements[scanner_idx]["circle"].get_visible():
                active_artists.append(radar_elements[scanner_idx]["circle"])
            active_artists.extend(
                [w for w in radar_elements[scanner_idx]["wedges"] if w.get_visible()]
            )
            # Add to active artists
            if radar_elements[scanner_idx]["circle"].get_visible():
                active_artists.append(radar_elements[scanner_idx]["circle"])
            active_artists.extend(
                [w for w in radar_elements[scanner_idx]["wedges"] if w.get_visible()]
            )

        # Clean up: ensure no core points remain in reachable set
        # This handles edge cases from concurrent updates
        scanner.confirmed_reachable_indices -= scanner.confirmed_core_indices

        # Update point colors
        scatter_plot.set_color(scanner.get_point_colors())

        # Return all active artists
        active_artists.extend(scanner.core_circles.values())
        return active_artists

    ani = FuncAnimation(
        fig,
        update_frame,
        frames=total_frames,
        interval=RADAR_STYLE["animation_interval"],
        blit=True,
        repeat=True,
    )
    return ani

In [ ]:
# Core connection animation configuration
CONNECTION_STYLE = {
    "animation_interval": 50,  # milliseconds between frames
    "link_draw_speed": 0.05,  # fraction of link drawn per frame
    "region_alpha": 0.15,  # alpha for semi-transparent regions
    "region_fade_in_frames": 20,  # frames to fade in regions
    "pause_frames": 10,  # pause frames between stages
}

In [ ]:
def core_connection_animation(
    ax,
    G,
    core_points,
    scan_radius,
    link_draw_speed=None,
    region_alpha=None,
    region_fade_in_frames=None,
    animation_interval=None,
    pause_frames=None,
):
    """
    Animate the process of connecting core points and showing their regions.
    All links are drawn simultaneously.

    Parameters:
    - ax: matplotlib Axes object to draw on
    - G: networkx Graph of core point connections
    - core_points: numpy array of core point coordinates
    - scan_radius: radius of core circles
    - link_draw_speed: speed of drawing links (0-1 per frame)
    - region_alpha: transparency of region circles
    - region_fade_in_frames: frames to fade in regions
    - animation_interval: milliseconds between frames
    - pause_frames: pause frames between stages

    Returns:
    - ani: FuncAnimation object
    """
    # Use configuration defaults if not specified
    link_draw_speed = (
        link_draw_speed
        if link_draw_speed is not None
        else CONNECTION_STYLE["link_draw_speed"]
    )
    region_alpha = (
        region_alpha if region_alpha is not None else CONNECTION_STYLE["region_alpha"]
    )
    region_fade_in_frames = (
        region_fade_in_frames
        if region_fade_in_frames is not None
        else CONNECTION_STYLE["region_fade_in_frames"]
    )
    animation_interval = (
        animation_interval
        if animation_interval is not None
        else CONNECTION_STYLE["animation_interval"]
    )
    pause_frames = (
        pause_frames if pause_frames is not None else CONNECTION_STYLE["pause_frames"]
    )

    fig = ax.figure

    # Get all edges to animate
    edges = list(G.edges())
    n_edges = len(edges)

    # Calculate frames - all links draw simultaneously
    frames_per_link = int(1 / link_draw_speed)
    total_link_frames = frames_per_link + pause_frames
    total_frames = total_link_frames + region_fade_in_frames

    # Create line objects for each edge
    line_objects = []
    for i, j in edges:
        (line,) = ax.plot(
            [],
            [],
            color=LINK_STYLE["color"],
            linewidth=LINK_STYLE["width"],
            linestyle=LINK_STYLE["linestyle"],
            zorder=2,
            visible=False,
        )
        line_objects.append(line)

    # Create region circles with compositing mode to prevent darkening
    region_circles = []
    for i, component in enumerate(nx.connected_components(G)):
        for core_idx in component:
            circle = Circle(
                core_points[core_idx],
                scan_radius,
                facecolor=LINK_STYLE["color"],
                edgecolor="none",
                alpha=0,
                zorder=1,
            )
            ax.add_patch(circle)
            region_circles.append(circle)

    def update_frame(frame):
        """Update function called for each animation frame."""
        active_artists = []

        # Phase 1: Draw all links simultaneously
        if frame < total_link_frames:
            # Calculate progress for all links
            if frame >= frames_per_link:
                # Pause phase - show all complete links
                progress = 1.0
            else:
                progress = frame / frames_per_link

            # Draw all links with the same progress
            for idx, (i, j) in enumerate(edges):
                p1, p2 = core_points[i], core_points[j]
                # Interpolate between start and end points
                x_data = [p1[0], p1[0] + (p2[0] - p1[0]) * progress]
                y_data = [p1[1], p1[1] + (p2[1] - p1[1]) * progress]
                line_objects[idx].set_data(x_data, y_data)
                line_objects[idx].set_visible(True)
                active_artists.append(line_objects[idx])

        # Phase 2: Fade in regions
        else:
            # Show all complete links
            for idx, (i, j) in enumerate(edges):
                p1, p2 = core_points[i], core_points[j]
                line_objects[idx].set_data([p1[0], p2[0]], [p1[1], p2[1]])
                line_objects[idx].set_visible(True)
                active_artists.append(line_objects[idx])

            # Fade in regions
            fade_frame = frame - total_link_frames
            fade_progress = min(fade_frame / region_fade_in_frames, 1.0)
            current_alpha = region_alpha * fade_progress

            for circle in region_circles:
                circle.set_alpha(current_alpha)
                active_artists.append(circle)

        return active_artists

    ani = FuncAnimation(
        fig,
        update_frame,
        frames=total_frames,
        interval=animation_interval,
        blit=True,
        repeat=True,
    )
    return ani

In [ ]:
def dbscan_workflow(
    n_points=50, scan_radius=10, min_points=3, canvas_width=150, canvas_height=100
):
    points = np.column_stack(
        (
            np.random.rand(n_points) * canvas_width,
            np.random.rand(n_points) * canvas_height,
        )
    )

    core_points, core_indices, points_in_circle = find_core_points(
        points, scan_radius, min_points
    )
    G = build_core_graph(core_indices, points_in_circle)
    reachable, noise = find_reachable_and_noise(
        points, core_indices, G, core_points, scan_radius
    )

    fig, axes = plt.subplots(5, 1, figsize=(10, 30))
    axes = axes.flatten()

    # Initial State: Original Data Distribution
    base_axis(axes[0], canvas_width, canvas_height)
    plot_all_points(axes[0], points)
    axes[0].set_title("Initial State: Original Data Distribution")
    add_dynamic_legend(axes[0])

    # Phase 1: ε-Neighborhood Search
    base_axis(axes[1], canvas_width, canvas_height)
    plot_all_points(axes[1], points)
    axes[1].set_title("Phase 1: ε-Neighborhood Search")
    add_dynamic_legend(axes[1], show_core_indices=True, show_reachable=True)
    ani = radar_animation(
        ax=axes[1],
        points=points,
        min_points=min_points,
        scan_radius=scan_radius,
    )
    animations.append(ani)

    # Intermediate State: Density Classification Result
    base_axis(axes[2], canvas_width, canvas_height)
    plot_all_points(axes[2], points, core_indices, reachable, noise)
    plot_core_circles(axes[2], core_points, scan_radius)
    axes[2].set_title("Intermediate State: Density Classification Result")
    add_dynamic_legend(
        axes[2],
        show_core_indices=True,
        show_circles=True,
        show_links=True,
        show_reachable=True,
        show_noise=True,
    )

    # Phase 2: Connecting Core Points to Form Clusters
    base_axis(axes[3], canvas_width, canvas_height)
    plot_all_points(axes[3], points, core_indices, reachable, noise)
    plot_core_circles(axes[3], core_points, scan_radius)
    axes[3].set_title("Phase 2: Connecting Core Points to Form Clusters")
    add_dynamic_legend(
        axes[3],
        show_core_indices=True,
        show_reachable=True,
        show_circles=True,
        show_links=True,
        show_regions=True,
        show_noise=True,
    )
    ani2 = core_connection_animation(
        ax=axes[3],
        G=G,
        core_points=core_points,
        scan_radius=scan_radius,
    )
    animations.append(ani2)

    # Final State: Connected Clusters & Noise
    base_axis(axes[4], canvas_width, canvas_height)
    plot_all_points(axes[4], points, core_indices, reachable, noise)
    plot_core_links(axes[4], G, core_points)
    plot_connected_regions(
        axes[4],
        G,
        core_points,
        scan_radius,
        fill=True,
        alpha=CONNECTION_STYLE["region_alpha"],
    )
    axes[4].set_title("Final State: Connected Clusters & Noise")
    add_dynamic_legend(
        axes[4], show_core_indices=True, show_reachable=True, show_noise=True
    )

    plt.tight_layout()
    plt.show()

In [ ]:
interact_manual(
    dbscan_workflow,
    n_points=IntSlider(min=10, max=200, step=10, value=50),
    scan_radius=FloatSlider(min=2, max=30, step=1, value=10),
    min_points=IntSlider(min=1, max=10, step=1, value=3),
    canvas_width=IntSlider(min=50, max=300, step=10, value=150),
    canvas_height=IntSlider(min=50, max=300, step=10, value=100),
)